# Mini-Project: 영화 댓글 감성 분류

> 3.2.6 장에 해당하는 코드

## 패키지 로딩, 데이터 전처리

In [1]:
# 코드 3-40

import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.data import Field, TabularDataset, Iterator
from konlpy.tag import Mecab
from train_utils_nlp import main

# 토큰화 함수로 MeCab 사용
tokenizer = Mecab()

# 필드 정의
TEXT = Field(sequential=True,
             use_vocab=True,
             tokenize=tokenizer.morphs,  
             lower=True, 
             batch_first=True)  
LABEL = Field(sequential=False,  
              use_vocab=False,   
              preprocessing = lambda x: int(x),
              batch_first=True, 
              is_target=True)

# 각 댓글에 해당하는 id, 사용하지 않지만 기본필드로 정의 해준다.
ID = Field(sequential=False,  
           use_vocab=False,   
           is_target=False)

# TabularDataset.splits 함수를 사용해 훈련 세트와 테스트 세트를 나눈다.
train_data, test_data = TabularDataset.splits(
    path='./data/nsmc', format='tsv', 
    train="ratings_train.txt",
    test="ratings_test.txt",
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
    skip_header=True)

# 단어장 생성
TEXT.build_vocab(train_data, min_freq=2)
# 데이터 개수 및 단어장 크기 확인
print("Train Data: {} / Test Data: {}".format(len(train_data), len(test_data)))
print("Vocab Size: {}".format(len(TEXT.vocab)))

# 환경 변수 설정
BATCH = 256  # 미니배치크기
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 디바이스
STEP = 10  # 총 반복스텝

# 데이터 로더 정의
train_loader = Iterator(dataset=train_data, batch_size=BATCH, device=DEVICE)
test_loader = Iterator(dataset=test_data, batch_size=BATCH, device=DEVICE)

Train Data: 150000 / Test Data: 50000
Vocab Size: 30046


# 영화 댓글 감성 분류 모델

In [2]:
# 코드 3-41

class SentimentCls(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, output_size,
                 num_layers=3, batch_first=True, bidirec=True):
        super(SentimentCls, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = num_layers
        self.n_direct = 2 if bidirec else 1
        self.embedding_layer = nn.Embedding(vocab_size, embed_size)
        self.rnn_layer = nn.LSTM(input_size=embed_size,
                                 hidden_size=hidden_size,
                                 num_layers=num_layers,
                                 batch_first=batch_first,
                                 bidirectional=bidirec)
        self.linear = nn.Linear(self.n_direct*hidden_size, output_size)

    def forward(self, x):
        embeded = self.embedding_layer(x)
        hidden, cell = self.init_hiddens(x.size(0), self.hidden_size, device=x.device)
        output, (hidden, cell) = self.rnn_layer(embeded, (hidden, cell))
        last_hidden = torch.cat([h for h in hidden[-self.n_direct:]], dim=1)
        scores = self.linear(last_hidden)
        return scores.view(-1)
    
    def init_hiddens(self, batch_size, hidden_size, device):
        hidden = torch.zeros(self.n_direct*self.n_layers, batch_size, hidden_size)
        cell = torch.zeros(self.n_direct*self.n_layers, batch_size, hidden_size)
        return hidden.to(device), cell.to(device)

## 모델, 손실함수 및 옵티마이저 선언

In [3]:
# 코드 3-42

# 모델 선언에 필요한 인자 설정
vocab_size = len(TEXT.vocab)  # V: 단어장 크기
embed_size = 128  # E: 임베딩 크기
hidden_size = 256  # D: 은닉층 크기
output_size = 1  # 출력층 크기
num_layers = 3  # RNN 층의 개수
batch_first = True  # RNN 입력의 첫번째 차원이 미니배치 크기인 경우 활성화
bidirec = True  # 양방향 순환 신경망 사용 여부

# 모델 선언
model = SentimentCls(vocab_size, embed_size, hidden_size, output_size,
                     num_layers, batch_first, bidirec).to(DEVICE)
# 매개변수 개수 확인하기
num_params = 0
for params in model.parameters():
    num_params += params.view(-1).size(0)
print("Total number of parameters: {}".format(num_params))

# 손실함수와 옵티마이저 선언
loss_function = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters())

Total number of parameters: 7790849


## 학습 및 테스트

In [4]:
# 코드 3-43

# 모델 훈련
main(model=model,
     train_loader=train_loader,
     test_loader=test_loader,
     loss_func=loss_function, 
     optimizer=optimizer, 
     n_step=STEP,
     save_path="./sent_cls.pt",
     print_step=256)

Train Step: 1 (00.00%)  	Loss: 0.6912
Train Step: 1 (43.69%)  	Loss: 0.4037
Train Step: 1 (87.38%)  	Loss: 0.3505
Test set: Average loss: 0.3567, Accuracy: 41889/50000 (83.78%)

Train Step: 2 (00.00%)  	Loss: 0.3177
Train Step: 2 (43.69%)  	Loss: 0.3178
Train Step: 2 (87.38%)  	Loss: 0.3586
Test set: Average loss: 0.3175, Accuracy: 43033/50000 (86.07%)
discard previous state, best model state saved!

Train Step: 3 (00.00%)  	Loss: 0.2356
Train Step: 3 (43.69%)  	Loss: 0.2237
Train Step: 3 (87.38%)  	Loss: 0.3137
Test set: Average loss: 0.3110, Accuracy: 43310/50000 (86.62%)
discard previous state, best model state saved!

Train Step: 4 (00.00%)  	Loss: 0.2397
Train Step: 4 (43.69%)  	Loss: 0.2003
Train Step: 4 (87.38%)  	Loss: 0.1971
Test set: Average loss: 0.3284, Accuracy: 43409/50000 (86.82%)
discard previous state, best model state saved!

Train Step: 5 (00.00%)  	Loss: 0.1402
Train Step: 5 (43.69%)  	Loss: 0.1344
Train Step: 5 (87.38%)  	Loss: 0.1412
Test set: Average loss: 0.3626

## 실제 댓글로 테스트 해보기

In [1]:
import torch
import torch.nn as nn
from senti_cls import SentimentCls
from torchtext.data import Field, TabularDataset, Iterator
from konlpy.tag import Mecab

## 전처리
# 토큰화 함수로 MeCab 사용
tokenizer = Mecab()

# 필드 정의
TEXT = Field(sequential=True,
             use_vocab=True,
             tokenize=tokenizer.morphs,  
             lower=True, 
             batch_first=True)  
LABEL = Field(sequential=False,  
              use_vocab=False,   
              preprocessing = lambda x: int(x),
              batch_first=True, 
              is_target=True)

# 각 댓글에 해당하는 id, 사용하지 않지만 기본필드로 정의 해준다.
ID = Field(sequential=False,  
           use_vocab=False,   
           is_target=False)

# TabularDataset.splits 함수를 사용해 훈련 세트와 테스트 세트를 나눈다.
train_data, test_data = TabularDataset.splits(
    path='./data/nsmc', format='tsv', 
    train="ratings_train.txt",
    test="ratings_test.txt",
    fields=[('id', ID), ('text', TEXT), ('label', LABEL)],
    skip_header=True)

# 단어장 생성
TEXT.build_vocab(train_data, min_freq=2)

## 모델 불러오기
# 모델 선언에 필요한 인자 설정
vocab_size = len(TEXT.vocab)  # V: 단어장 크기
embed_size = 128  # E: 임베딩 크기
hidden_size = 256  # D: 은닉층 크기
output_size = 1  # 출력층 크기
num_layers = 3  # RNN 층의 개수
batch_first = True  # RNN 입력의 첫번째 차원이 미니배치 크기인 경우 활성화
bidirec = True  # 양방향 순환 신경망 사용 여부
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'  # 디바이스

# 모델 선언 및 불러오기
model = SentimentCls(vocab_size, embed_size, hidden_size, output_size,
                     num_layers, batch_first, bidirec).to(DEVICE)
model.load_state_dict(torch.load("./sent_cls.pt"))
print("Load Complete!")

# 실제 테스트
def test_input(model, field, tokenizer, device):
    sentence = input("테스트할 댓글 작성: ")
    x = field.process([tokenizer.morphs(sentence)]).to(device)
    output = model(x)
    pred = torch.sigmoid(output).ge(0.5).item()
    print("---결과---")
    if pred == 1:
        print("긍정")
    else:
        print("부정")
        
test_input(model, field=TEXT, tokenizer=tokenizer, device=DEVICE)

Load Complete!


테스트할 댓글 작성:  이 영화 정말 재밌어요 ㅋㅋㅋ


---결과---
긍정
